In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp2-spam/train.csv
/kaggle/input/2021-ml-tp2-spam/test.csv
/kaggle/input/2021-ml-tp2-spam/sample_submit.csv


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [3]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df_train = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/train.csv",encoding="latin-1")
df_test = pd.read_csv("/kaggle/input/2021-ml-tp2-spam/test.csv",encoding="latin-1")
df_train.Data

0       No I'm in the same boat. Still here at my moms...
1       (Bank of Granite issues Strong-Buy) EXPLOSIVE ...
2          They r giving a second chance to rahul dengra.
3          O i played smash bros  &lt;#&gt;  religiously.
4       PRIVATE! Your 2003 Account Statement for 07973...
                              ...                        
4452    I came hostel. I m going to sleep. Plz call me...
4453                               Sorry, I'll call later
4454        Prabha..i'm soryda..realy..frm heart i'm sory
4455                           Nt joking seriously i told
4456                  In work now. Going have in few min.
Name: Data, Length: 4457, dtype: object

In [5]:
X_train = df_train["Data"]
y_train=df_train["Class"]
X_test=df_test["Data"]

In [6]:
# ------------------------------------------------
# [Empty Module #1] 텍스트 데이터 전처리
# ------------------------------------------------
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer

def data_processing(text):
    # ------------------------------------------------------------
    # 구현 가이드라인 
    # ------------------------------------------------------------
    # [1] re.sub 사용해 text 속 '[^A-Za-z]' 외의 문자만을 찾아내 제거한후, pre_words 변수에 저장
    #      1) pattern은 '[^A-Za-z]', repl=' ' 로 각각 설정.
    #      2) 이모지나 숫자,점과 같은 문자외의 것들을 제거했다. (이모지는 감정 분석과 관련해서 몇가지 의미를 나타내지만 이 테스크에서는 이러한 의미도 제거함.)
    #
    # [2] pre_words의 lower 내장 함수를 이용해 대문자들은 소문자로 변경
    #      1)  대, 소문자가 구분되어 있으면 "Go"와 "go" 와 같이 동일한 단어를 머신은 다른 단어로 취급한다. 따라서 대문자를 모두 소문자로 변경.
    #  
    # [3] word_tokenize 함수를 이용해 pre_word 를 토큰화하여 word를 리스트화한 후 tokenized_words변수에 저장
    #
    # [4] nltk 라이브러리로 다운 받은 stopwords의 "words" 내장 함수를 이용해 english 불용어를 찾아서 stops 변수에 저장  
    #      1) 불용어: 텍스트 분류에서 불용어는 텍스트의 중요도을 결정하는데 영향을 미치지 않는 단어임. 
    #                    (ex: the, we, a , will), 따라서 불필요한 단어가 예측 모델에 악영향을 끼칠 수 있기 때문에 제거.
    #
    # [5] [3] 에서 찾은 문자열 중 단어가 [4] 에서 찾은 불용어 속에 없을 경우, tokenized_words_remove 리스트에 append 
    #
    # [6] PorterStemmer 속 stem 내장 함수를 이용해 동일 의미를 갖는 단어를 동일한 단어로 변경하는 과정을 거친 후 다시 저장.
    #    
    # ------------------------------------------------------------
    ##############
    
    # [1]
    pre_words = re.sub('[^A-Za-z]', ' ', text)
    ##############
   
    # [2]
    pre_words = pre_words.lower()
    
    ##############
    
    # [3]
    tokenized_words = word_tokenize(pre_words)
    ##############
    
    # [4]
    stops = stopwords.words('english')
    
    ##############
    tokenized_words_remove=[]
    
    for w in tokenized_words: 
        # [5]
        if w not in stops:
            tokenized_words_remove.append(w)
        ##############
    
    tokenized_words = tokenized_words_remove
    ############
    
    stemmer = PorterStemmer()
    for i in range(len(tokenized_words)):
        # [6]
        tokenized_words[i] = stemmer.stem(tokenized_words[i])
        
        ##############
    
    return ( " ".join( tokenized_words ))

In [7]:
X_train=[data_processing(i) for i in X_train]
X_test=[data_processing(i) for i in X_test]

In [8]:
# ------------------------------------------------
# [Empty Module #2] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.feature_extraction.text import CountVectorizer
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  CountVectorizer를 정의 
#           1) max_features를 100으로 지정 
# [2] X_train 과 X_test를 numpy array로 변환 후 데이터 타입을 "U"로 변경해 저장
#
# [3] CountVectorizer를 이용해 X_train은 학습 및 변환을 하고, X_test는 변환을 진행. 
# ------------------------------------------------------------
###########
from sklearn.feature_extraction.text import CountVectorizer
# [1]
vectorizer = CountVectorizer(max_features = 100)
             
# [2]
X_train_arr = np.array(X_train)
X_test_arr = np.array(X_test)

X_train_arr = X_train_arr.astype('U')
X_test_arr = X_test_arr.astype('U')

# [3]
X_train_features = vectorizer.fit_transform(X_train_arr)
X_test_features = vectorizer.transform(X_test_arr)

In [9]:
y_train[y_train=="ham"]=0
y_train[y_train=="spam"]=1
y_train=y_train.astype("uint8")

In [10]:
# ------------------------------------------------
# [Empty Module #3] 텍스트 데이터 Bag of word  feature  화 
# ------------------------------------------------
from sklearn.svm import SVC
# ------------------------------------------------------------
# 구현 가이드라인 
# ------------------------------------------------------------
# [1]  SVC 정의 
#           1) gamma="auto" 사용 
# [2] X_train_features과 y_train으로 SVC 학습진행 후, X_test_features로 predict 진행
#
# ------------------------------------------------------------
###########




# [1]
svc=SVC(gamma = 'auto')

# [2]
svc.fit(X_train_features, y_train)
y_pred = svc.predict(X_test_features)

In [11]:
count = 0
for i in y_pred:
    if i ==0:
        count +=1

count

1025

In [12]:
df_pred={"ID": range(np.array(y_pred).shape[0]),"Class":y_pred}
df_pred=pd.DataFrame(df_pred)
df_pred.to_csv("predict.csv",index=False)